# Cleaning methodology for pollutant data

This notebook contains the key steps taken to gather and clean air pollution data. As the data set is quite large and can only be downloaded in small chunks, we are currently investigation different ways of downloading the data in order to increase efficiency. As we continue to explore the data, more substantial filtering and cleaning will take place. 

In [7]:
import pandas as pd
import numpy as np

## Step 1: Identify boundary for monitoring. 

Having manually sifted through the data, we were able to split several monitoring sites into "inner" and "outer" ranges based on their geographic location. The data points closest to our area of focus (Heathrow Airport) have been categorised as "inner" locations and will be vital in our investigation into emissions. The "outer" locations will be used as a way of comparisson to gain a deeper understanding of the scale of the impact arising from air pollution. 

|CCG | Borough| Inner location - monitoring station| Outer location - monitoring site|
|---:|:-----|:-----------|:------------|
|Hillingdon |Hillingdon| Hillingdon South Ruislip, Hillingdon 2 Hillingdon Hospital, Hillingdon Oxford Avenue, Hillingdon Harmondsworth, Hillingdon Harmondsworth Osiris, Hillingdon Hayes, Heathrow LHR2, Heathrow Bath Road, Hillingdon Sipson, Heathrow Green Gates| |
|East Berkshire | Slough |Slough Town Centre Wellington Street,Slough Brands Hill London Road,Slough Windmill Bath Road,Slough Colnbrook,Slough Town Centre A4,Slough Lakeside 1 Osiris,Slough Colnbrook Osiris,Slough Chalvey,Slough Lakeside 2,Slough Lakeside 2 Osiris,Slough - Dennis Way LP11,Slough - Monksfield Way LP20,Slough - The Hawthorns LP2,Slough - Erica Close LP3,Slough - Hatton Avenue LP13,Slough - St Andrews Way LP12,Slough - The Hawthorns LP10,Slough - Francis Way LP13,Slough - The Hawthorns LP1,Slough - Monksfield Way LP19,Slough - Brighton Spur LP3,Slough - Bower Way LP1,Slough - Hatton Avenue LP3,Slough - Cinder Track LP37|
|Hounslow| Hounslow | Hounslow Cranford, Hounslow Chiswick, Hounslow Brentford, Hounslow Heston, Hounslow Hatton Cross, Hounslow Feltham, Hounslow Gunnersbury |
|Ealing | Ealing | Ealing Horn Lane
|Buckinghamshire|South Bucks|Iver Thorney Lane North, Iver North Park Road, Iver Primary School|
|Surrey Heartlands|Richmond|- |Elmbridge|
| - | Spelthorne | Spelthorne Shepperton Squire's Bridge Road, Spelthorne knowle Green, Spelthorne Sunbury Cross, Heathrow Oaks Road|
| - |Waverly and Woking | - | H&F Shepherd’s Bush, Godalming Ockford Road 2|
|South West London| Richmond | London Teddington Bushy Park. |
|Hammersmith & Fulham|London Borough of Hammersmith and Fulham|-| H&F Hammersmith Town Centre, H&F Shepherd’s Bush|
|Watford|Hertfordshire and Bedfordshire|-|Watford Town Hall|
|Oxfordshire|Oxfordshire|-|Oxford High St, Oxford St Ebbes (Cal Club), Oxford Center Roadside, Oxford St Ebbes|
| Berkshire West| Readiing| - | Reading Caversham Road, Reading Oxford Road, Reading London Road, Reading New Town|

## Step 2: Format the Data

In [8]:
boroughs = ['ealing', 'hounslow', 'hillingdon', 'richmond', 'slough', 'spelthorne']

In [9]:
li = []

for borough in boroughs:
    df = pd.read_excel('./raw_data/{}.xlsx'.format(borough), header=[0,1], sheet_name=None)
    df = pd.concat(df.values(), axis=0)
    df = pd.DataFrame(df, columns=df.keys())
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [10]:
# frame.to_excel("concatinated_pollution_data.xlsx")

In [11]:
# df = pd.read_excel('concatinated_pollution_data.xlsx', header=[0,1], sheet_name=None)
# df = pd.concat(df.values(), axis=0)
# df = pd.DataFrame(df, columns=df.keys())
# df.head()

In [12]:
# pd.set_option('display.max_columns', 100)

## Step 3: Unify missing data

In [31]:
df = frame.replace('No data', np.nan)
df = df.replace('No Data', np.nan)
df.head()

## Step 4: Identify closed monitoring stations

In [14]:
non_null_columns = [col for col in df.columns if df.loc[:, col].notna().any()]
open_monitoring_sites = df[non_null_columns]
open_monitoring_sites

Hounslow 2 - Cranford                                     \
                 Nitric Oxide Nitrogen dioxide Non-volatile PM10   
0                         NaN              NaN               NaN   
1                         NaN              NaN               NaN   
2                         NaN              NaN               NaN   
3                         NaN              NaN               NaN   
4                         NaN              NaN               NaN   
...                       ...              ...               ...   
915754                    NaN              NaN               NaN   
915755                    NaN              NaN               NaN   
915756                    NaN              NaN               NaN   
915757                    NaN              NaN               NaN   
915758                    NaN              NaN               NaN   

                                                                     \
       Non-volatile PM2.5 Oxides of Nitrogen Ozone PM1 Particulates   
0                     NaN                NaN   NaN              NaN   
1                     NaN                NaN   NaN              NaN   
2                     NaN                NaN   NaN              NaN   
3                     NaN                NaN   NaN              NaN   
4                     NaN                NaN   NaN              NaN   
...                   ...                ...   ...              ...   
915754                NaN                NaN   NaN              NaN   
915755                NaN                NaN   NaN              NaN   
915756                NaN                NaN   NaN              NaN   
915757                NaN                NaN   NaN              NaN   
915758                NaN                NaN   NaN              NaN   

                                                                   ...  \
       PM10 Particulate matter PM2.5 Particulate matter Status.10  ...   
0                          NaN                      NaN       NaN  ...   
1                          NaN                      NaN       NaN  ...   
2                          NaN                      NaN       NaN  ...   
3                          NaN                      NaN       NaN  ...   
4                          NaN                      NaN       NaN  ...   
...                        ...                      ...       ...  ...   
915754                     NaN                      NaN       NaN  ...   
915755                     NaN                      NaN       NaN  ...   
915756                     NaN                      NaN       NaN  ...   
915757                     NaN                      NaN       NaN  ...   
915758                     NaN                      NaN       NaN  ...   

       Slough Windmill Bath Road          Spelthorne Sunbury Cross  \
                          Status Status.1         Nitrogen dioxide   
0                            NaN      NaN                      NaN   
1                            NaN      NaN                      NaN   
2                            NaN      NaN                      NaN   
3                            NaN      NaN                      NaN   
4                            NaN      NaN                      NaN   
...                          ...      ...                      ...   
915754                       NaN      NaN                      NaN   
915755                       NaN      NaN                      NaN   
915756                       NaN      NaN                      NaN   
915757                       NaN      NaN                      NaN   
915758                       NaN      NaN                      NaN   

                                                  \
       PM10 particulate matter (Hourly measured)   
0                                            NaN   
1                                            NaN   
2                                            NaN   
3                                            NaN   
4            

## Step 5: Setting Date and Time columns to Datetime

In [17]:
open_monitoring_sites[('Unnamed: 1_level_0','Time')] = [str(x)[-8:] for x in open_monitoring_sites[('Unnamed: 1_level_0','Time')]]

/Users/panchals/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
open_monitoring_sites

Hounslow 2 - Cranford                                     \
                 Nitric Oxide Nitrogen dioxide Non-volatile PM10   
0                         NaN              NaN               NaN   
1                         NaN              NaN               NaN   
2                         NaN              NaN               NaN   
3                         NaN              NaN               NaN   
4                         NaN              NaN               NaN   
...                       ...              ...               ...   
915754                    NaN              NaN               NaN   
915755                    NaN              NaN               NaN   
915756                    NaN              NaN               NaN   
915757                    NaN              NaN               NaN   
915758                    NaN              NaN               NaN   

                                                                     \
       Non-volatile PM2.5 Oxides of Nitrogen Ozone PM1 Particulates   
0                     NaN                NaN   NaN              NaN   
1                     NaN                NaN   NaN              NaN   
2                     NaN                NaN   NaN              NaN   
3                     NaN                NaN   NaN              NaN   
4                     NaN                NaN   NaN              NaN   
...                   ...                ...   ...              ...   
915754                NaN                NaN   NaN              NaN   
915755                NaN                NaN   NaN              NaN   
915756                NaN                NaN   NaN              NaN   
915757                NaN                NaN   NaN              NaN   
915758                NaN                NaN   NaN              NaN   

                                                                   ...  \
       PM10 Particulate matter PM2.5 Particulate matter Status.10  ...   
0                          NaN                      NaN       NaN  ...   
1                          NaN                      NaN       NaN  ...   
2                          NaN                      NaN       NaN  ...   
3                          NaN                      NaN       NaN  ...   
4                          NaN                      NaN       NaN  ...   
...                        ...                      ...       ...  ...   
915754                     NaN                      NaN       NaN  ...   
915755                     NaN                      NaN       NaN  ...   
915756                     NaN                      NaN       NaN  ...   
915757                     NaN                      NaN       NaN  ...   
915758                     NaN                      NaN       NaN  ...   

       Slough Windmill Bath Road          Spelthorne Sunbury Cross  \
                          Status Status.1         Nitrogen dioxide   
0                            NaN      NaN                      NaN   
1                            NaN      NaN                      NaN   
2                            NaN      NaN                      NaN   
3                            NaN      NaN                      NaN   
4                            NaN      NaN                      NaN   
...                          ...      ...                      ...   
915754                       NaN      NaN                      NaN   
915755                       NaN      NaN                      NaN   
915756                       NaN      NaN                      NaN   
915757                       NaN      NaN                      NaN   
915758                       NaN      NaN                      NaN   

                                                  \
       PM10 particulate matter (Hourly measured)   
0                                            NaN   
1                                            NaN   
2                                            NaN   
3                                            NaN   
4            

In [19]:
open_monitoring_sites[('Unnamed: 1_level_0','Time')] = pd.to_datetime(open_monitoring_sites[('Unnamed: 1_level_0','Time')], format='%H:%M:%S').dt.strftime("%H:%M:%S")

/Users/panchals/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
open_monitoring_sites.columns

MultiIndex([(   ' Hounslow 2 - Cranford', ...),
            (   ' Hounslow 2 - Cranford', ...),
            (   ' Hounslow 2 - Cranford', ...),
            (   ' Hounslow 2 - Cranford', ...),
            (   ' Hounslow 2 - Cranford', ...),
            (   ' Hounslow 2 - Cranford', ...),
            (   ' Hounslow 2 - Cranford', ...),
            (   ' Hounslow 2 - Cranford', ...),
            (   ' Hounslow 2 - Cranford', ...),
            (   ' Hounslow 2 - Cranford', ...),
            ...
            ('Slough Windmill Bath Road', ...),
            ('Slough Windmill Bath Road', ...),
            ( 'Spelthorne Sunbury Cross', ...),
            ( 'Spelthorne Sunbury Cross', ...),
            ( 'Spelthorne Sunbury Cross', ...),
            ( 'Spelthorne Sunbury Cross', ...),
            ( 'Spelthorne Sunbury Cross', ...),
            ( 'Spelthorne Sunbury Cross', ...),
            (       'Unnamed: 0_level_0', ...),
            (       'Unnamed: 1_level_0', ...)],
           length=274)

In [21]:
open_monitoring_sites.set_index([('Unnamed: 0_level_0','Date'), ('Unnamed: 1_level_0','Time')], inplace=True)
open_monitoring_sites

Hounslow 2 - Cranford  \
                                                                Nitric Oxide   
(Unnamed: 0_level_0, Date) (Unnamed: 1_level_0, Time)                          
2004-01-01                 01:00:00                                      NaN   
                           02:00:00                                      NaN   
                           03:00:00                                      NaN   
                           04:00:00                                      NaN   
                           05:00:00                                      NaN   
...                                                                      ...   
2020-12-01                 20:00:00                                      NaN   
                           21:00:00                                      NaN   
                           22:00:00                                      NaN   
                           23:00:00                                      NaN   
                           00:00:00                                      NaN   

                                                                        \
                                                      Nitrogen dioxide   
(Unnamed: 0_level_0, Date) (Unnamed: 1_level_0, Time)                    
2004-01-01                 01:00:00                                NaN   
                           02:00:00                                NaN   
                           03:00:00                                NaN   
                           04:00:00                                NaN   
                           05:00:00                                NaN   
...                                                                ...   
2020-12-01                 20:00:00                                NaN   
                           21:00:00                                NaN   
                           22:00:00                                NaN   
                           23:00:00                                NaN   
                           00:00:00                                NaN   

                                                                         \
                                                      Non-volatile PM10   
(Unnamed: 0_level_0, Date) (Unnamed: 1_level_0, Time)                     
2004-01-01                 01:00:00                                 NaN   
                           02:00:00                                 NaN   
                           03:00:00                                 NaN   
                           04:00:00                                 NaN   
                           05:00:00                                 NaN   
...                                                                 ...   
2020-12-01                 20:00:00                                 NaN   
                           21:00:00                                 NaN   
                           22:00:00                                 NaN   
                           23:00:00                                 NaN   
                           00:00:00                                 NaN   

                                                                          \
                                                      Non-volatile PM2.5   
(Unnamed: 0_level_0, Date) (Unnamed: 1_level_0, Time)                      
2004-01-01                 01:00:00                                  NaN   
                           02:00:00                                  NaN   
                           03:00:00                                  NaN   
                           04:00:00                                  NaN   
                           05:00:00                                  NaN   
...                                                                  ...   
2020-12-01                 20:00:00                                  NaN   
                           21:00:00                                  NaN   
          

In [23]:
open_monitoring_sites = open_monitoring_sites.rename_axis(['Date','Time'])
open_monitoring_sites

Hounslow 2 - Cranford                                     \
                              Nitric Oxide Nitrogen dioxide Non-volatile PM10   
Date       Time                                                                 
2004-01-01 01:00:00                    NaN              NaN               NaN   
           02:00:00                    NaN              NaN               NaN   
           03:00:00                    NaN              NaN               NaN   
           04:00:00                    NaN              NaN               NaN   
           05:00:00                    NaN              NaN               NaN   
...                                    ...              ...               ...   
2020-12-01 20:00:00                    NaN              NaN               NaN   
           21:00:00                    NaN              NaN               NaN   
           22:00:00                    NaN              NaN               NaN   
           23:00:00                    NaN              NaN               NaN   
           00:00:00                    NaN              NaN               NaN   

                                                                 \
                    Non-volatile PM2.5 Oxides of Nitrogen Ozone   
Date       Time                                                   
2004-01-01 01:00:00                NaN                NaN   NaN   
           02:00:00                NaN                NaN   NaN   
           03:00:00                NaN                NaN   NaN   
           04:00:00                NaN                NaN   NaN   
           05:00:00                NaN                NaN   NaN   
...                                ...                ...   ...   
2020-12-01 20:00:00                NaN                NaN   NaN   
           21:00:00                NaN                NaN   NaN   
           22:00:00                NaN                NaN   NaN   
           23:00:00                NaN                NaN   NaN   
           00:00:00                NaN                NaN   NaN   

                                                              \
                    PM1 Particulates PM10 Particulate matter   
Date       Time                                                
2004-01-01 01:00:00              NaN                     NaN   
           02:00:00              NaN                     NaN   
           03:00:00              NaN                     NaN   
           04:00:00              NaN                     NaN   
           05:00:00              NaN                     NaN   
...                              ...                     ...   
2020-12-01 20:00:00              NaN                     NaN   
           21:00:00              NaN                     NaN   
           22:00:00              NaN                     NaN   
           23:00:00              NaN                     NaN   
           00:00:00              NaN                     NaN   

                                                        ...  \
                    PM2.5 Particulate matter Status.10  ...   
Date       Time                                         ...   
2004-01-01 01:00:00                      NaN       NaN  ...   
           02:00:00                      NaN       NaN  ...   
           03:00:00                      NaN       NaN  ...   
           04:00:00                      NaN       NaN  ...   
           05:00:00                      NaN       NaN  ...   
...                                      ...       ...  ...   
2020-12-01 20:00:00                      NaN       NaN  ...   
           21:00:00                      NaN       NaN  ...   
           22:00:00                      NaN       NaN  ...   
           23:00:00                      NaN       NaN  ...   
           00:00:00                      NaN       NaN  ...   

                    Slough Windmill Bath Road  \
                             Nitrogen dioxide   
Date       Time                                 
2004-01-01 01:00:00 

## Step 6: Removing Multi-tier Columns

In [40]:
open_monitoring_sites_stack = open_monitoring_sites.stack(0, dropna=True).rename_axis(('Date', 'Time','Location'))
open_monitoring_sites_stack

Carbon monoxide  \
Date       Time     Location                                         
2004-01-01 01:00:00 Ealing - Acton Town Hall               0.81494   
                    Ealing - Ealing Town Hall                  NaN   
                    Ealing - Hanger Lane Gyratory              NaN   
           02:00:00 Ealing - Acton Town Hall               0.93136   
                    Ealing - Ealing Town Hall                  NaN   
...                                                            ...   
2020-12-01 22:00:00 Godalming Ockford Road 2                   NaN   
           23:00:00 Farnham The Woolmead                       NaN   
                    Godalming Ockford Road 2                   NaN   
           00:00:00 Farnham The Woolmead                       NaN   
                    Godalming Ockford Road 2                   NaN   

                                                  Nitric Oxide  \
Date       Time     Location                                     
2004-01-01 01:00:00 Ealing - Acton Town Hall               NaN   
                    Ealing - Ealing Town Hall              NaN   
                    Ealing - Hanger Lane Gyratory          NaN   
           02:00:00 Ealing - Acton Town Hall               NaN   
                    Ealing - Ealing Town Hall              NaN   
...                                                        ...   
2020-12-01 22:00:00 Godalming Ockford Road 2               NaN   
           23:00:00 Farnham The Woolmead                   NaN   
                    Godalming Ockford Road 2               NaN   
           00:00:00 Farnham The Woolmead                   NaN   
                    Godalming Ockford Road 2               NaN   

                                                  Nitrogen dioxide  \
Date       Time     Location                                         
2004-01-01 01:00:00 Ealing - Acton Town Hall                 35.19   
                    Ealing - Ealing Town Hall              18.5512   
                    Ealing - Hanger Lane Gyratory          46.8563   
           02:00:00 Ealing - Acton Town Hall                17.595   
                    Ealing - Ealing Town Hall              11.0925   
...                                                            ...   
2020-12-01 22:00:00 Godalming Ockford Road 2               37.8524   
           23:00:00 Farnham The Woolmead                   31.9377   
                    Godalming Ockford Road 2                34.697   
           00:00:00 Farnham The Woolmead                    29.141   
                    Godalming Ockford Road 2               30.9496   

                                                  Non-volatile PM10  \
Date       Time     Location                                          
2004-01-01 01:00:00 Ealing - Acton Town Hall                    NaN   
                    Ealing - Ealing Town Hall                   NaN   
                    Ealing - Hanger Lane Gyratory               NaN   
           02:00:00 Ealing - Acton Town Hall                    NaN   
                    Ealing - Ealing Town Hall                   NaN   
...                                                             ...   
2020-12-01 22:00:00 Godalming Ockford Road 2                    NaN   
           23:00:00 Farnham The Woolmead                        NaN   
                    Godalming Ockford Road 2                    NaN   
           00:00:00 Farnham The Woolmead                        NaN   
                    Godalming Ockford Road 2                    NaN   

                                                  Non-volatile PM2.5  \
Date       Time     Location                                           
2004-01-01 01:00:00 Ealing - Acton Town Hall                     NaN   
                    Ealing - Ealing Town Hall                    NaN   
                    Ealing - Hanger Lane Gyratory                NaN   
           02:00:00 Ealing - Acton Town Hall                     NaN   
              

In [41]:
open_monitoring_sites_stack.reset_index(level=['Location'])

Location  Carbon monoxide  \
Date       Time                                                       
2004-01-01 01:00:00       Ealing - Acton Town Hall          0.81494   
           01:00:00      Ealing - Ealing Town Hall              NaN   
           01:00:00  Ealing - Hanger Lane Gyratory              NaN   
           02:00:00       Ealing - Acton Town Hall          0.93136   
           02:00:00      Ealing - Ealing Town Hall              NaN   
...                                            ...              ...   
2020-12-01 22:00:00       Godalming Ockford Road 2              NaN   
           23:00:00           Farnham The Woolmead              NaN   
           23:00:00       Godalming Ockford Road 2              NaN   
           00:00:00           Farnham The Woolmead              NaN   
           00:00:00       Godalming Ockford Road 2              NaN   

                    Nitric Oxide Nitrogen dioxide Non-volatile PM10  \
Date       Time                                                       
2004-01-01 01:00:00          NaN            35.19               NaN   
           01:00:00          NaN          18.5512               NaN   
           01:00:00          NaN          46.8563               NaN   
           02:00:00          NaN           17.595               NaN   
           02:00:00          NaN          11.0925               NaN   
...                          ...              ...               ...   
2020-12-01 22:00:00          NaN          37.8524               NaN   
           23:00:00          NaN          31.9377               NaN   
           23:00:00          NaN           34.697               NaN   
           00:00:00          NaN           29.141               NaN   
           00:00:00          NaN          30.9496               NaN   

                    Non-volatile PM2.5 Oxides of Nitrogen     Ozone  \
Date       Time                                                       
2004-01-01 01:00:00                NaN                NaN       NaN   
           01:00:00                NaN                NaN  39.91400   
           01:00:00                NaN                NaN       NaN   
           02:00:00                NaN                NaN       NaN   
           02:00:00                NaN                NaN  54.48261   
...                                ...                ...       ...   
2020-12-01 22:00:00                NaN                NaN       NaN   
           23:00:00                NaN                NaN       NaN   
           23:00:00                NaN                NaN       NaN   
           00:00:00                NaN                NaN       NaN   
           00:00:00                NaN                NaN       NaN   

                    PM1 Particulates PM10 Particulate matter  ...  Status.2  \
Date       Time                                               ...             
2004-01-01 01:00:00              NaN                     NaN  ...   V mgm-3   
           01:00:00              NaN                     NaN  ...   V ugm-3   
           01:00:00              NaN                     NaN  ...       NaN   
           02:00:00              NaN                     NaN  ...   V mgm-3   
           02:00:00              NaN                     NaN  ...   V ugm-3   
...                              ...                     ...  ...       ...   
2020-12-01 22:00:00              NaN                     NaN  ...       NaN   
           23:00:00              NaN                     NaN  ...       NaN   
           23:00:00              NaN                     NaN  ...       NaN   
           00:00:00              NaN                     NaN  ...       NaN   
           00:00:00              NaN                     NaN  ...       NaN   

                          Status.3 Status.4  Status.5 Status.6 Status.7  \
Date       Time                                                           
2004-01-01 01:00:00  V ugm-3 (VCM)  V ugm-3       NaN      NaN      NaN   
           01:00:

## Renaming status columns

In [42]:
list(open_monitoring_sites_stack.columns)

['Carbon monoxide',
 'Nitric Oxide',
 'Nitrogen dioxide',
 'Non-volatile PM10',
 'Non-volatile PM2.5',
 'Oxides of Nitrogen',
 'Ozone',
 'PM1 Particulates',
 'PM10 Particulate matter',
 'PM10 particulate matter (Hourly measured)',
 'PM2.5 Particulate matter',
 'PM2.5 Particulate matter.1',
 'PM2.5 particulate matter (Hourly measured)',
 'Status',
 'Status.1',
 'Status.10',
 'Status.12',
 'Status.14',
 'Status.2',
 'Status.3',
 'Status.4',
 'Status.5',
 'Status.6',
 'Status.7',
 'Status.8',
 'Status.9',
 'Sulphur dioxide',
 'Volatile PM10']

In [43]:
pollutants = ['Carbon monoxide',
 'Nitric Oxide',
 'Nitrogen dioxide',
 'Non-volatile PM10',
 'Non-volatile PM2.5',
 'Oxides of Nitrogen',
 'Ozone',
 'PM1 Particulates',
 'PM10 Particulate matter',
 'PM10 particulate matter (Hourly measured)',
 'PM2.5 Particulate matter',
 'PM2.5 Particulate matter.1',
 'PM2.5 particulate matter (Hourly measured)',
 'Sulphur dioxide',
 'Volatile PM10']

In [44]:
units = {'Status': 'Carbon monoxide units',
 'Status.1' : 'Nitric Oxide units',
 'Status.10' :'Nitrogen dioxide units',
 'Status.12' :'Non-volatile PM10 units',
 'Status.14' : 'Non-volatile PM2.5 units',
 'Status.2' : 'Oxides of Nitrogen units',
 'Status.3' : 'Ozone units',
 'Status.4' : 'PM1 Particulates units',
 'Status.5' : 'PM10 Particulate matter units',
 'Status.6' : 'PM10 particulate matter (Hourly measured) units',
 'Status.7' : 'PM2.5 Particulate matter units',
 'Status.8' : 'PM2.5 Particulate matter.1 units',
 'Status.9' : 'PM2.5 particulate matter (Hourly measured) units'
 }

In [38]:
# open_monitoring_sites_stack.columns = open_monitoring_sites_stack.columns.map(units)

In [26]:
open_monitoring_sites_stack.to_csv("stacked_inner_pollution_data.csv")

In [1]:
# clean_data = pd.read_excel('concatinated_pollution_data.xlsx', header=[0,1], sheet_name=None)
# clean_dataclean_data = pd.concat(clean_data.values(), axis=0)
# clean_data = pd.DataFrame(clean_data, columns=clean_data.keys())
# clean_data.head()

## Next Steps

- Use API call to gather data from monitoring stations
- Investigate other data sources for mass downloads and ease
- Aggregate data on a daily basis
- Investigate similarities/differences between outer and inner locations, perform EDA on each
- Aggregate data annually and merge with Health and Flight data
